In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "nOVDwpTbqq375XS6RP3TpAB6xawPQRma"

# Get Access Token

In [11]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "mV3n2jYY2q0GD6OfIxCT6Mt0EzbkEf6s"
kite.set_access_token(access_token)
print(access_token)

mV3n2jYY2q0GD6OfIxCT6Mt0EzbkEf6s


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BFO', 'CDS', 'BSE', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# NOTE:
# After every request ensure that you sleep for 0.5 second because there is a limit of a maximum of  3 requests per second.

# Placing a trade

In [13]:
def place_order(trading_symbol, exchange, qty, transaction_type, product, order_type, variety='regular',price=None,validity=None,disclosed_quantity=None,trigger_price=None,squareoff=None,stoploss=None,trailing_stoploss=None, tag=None):
    """
    Refer: https://kite.trade/docs/connect/v3/orders/
    
    tradingsymbol Eg RELIANCE, 
    exchange => ['MF', 'NSE', 'NFO', 'CDS', 'BSE', 'BFO'],
    quantity as integer,
    variety is 'regular' by default,
    transaction_type => 'BUY' or 'SELL',
    product Eg 'CNC' for Equity,
    order_type => ['MARKET', 'LIMIT', 'SL', 'SL-M'],
    price=None,
    validity=None,
    disclosed_quantity=None,
    trigger_price=None,
    squareoff=None,
    stoploss=None,
    trailing_stoploss=None,
    tag=None
    """
    order_id = kite.place_order(tradingsymbol=trading_symbol, 
                     exchange=exchange,
                     quantity=qty,
                     variety=variety,
                     transaction_type=transaction_type,
                     product=product,
                     order_type=order_type,
                     price=price,
                     validity=validity,
                     disclosed_quantity=disclosed_quantity,
                     trigger_price=trigger_price,
                     squareoff=squareoff,
                     stoploss=stoploss,
                     trailing_stoploss=trailing_stoploss,
                     tag=tag)
    return order_id

# Get Instrument Tokens

In [14]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [15]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

In [16]:
instoken = get_instrument_token('BANKNIFTY20OCTFUT','FUT','NFO')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 865177


# Get Exponential Moving Average

In [17]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [18]:
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

In [19]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# BACKTEST TRADING PLATFORM

In [20]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_transaction_type, current_status

#### 3 minute interval

In [124]:
market_open = datetime(2020, 8, 14, 9, 30)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='3minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:29:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A30%3A00&to=2020-08-14+09%3A30%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2522
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A31%3A00&to=2020-08-14+09%3A31%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2516
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A32%3A00&to=2020-08-14+09%3A32%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2518
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A56%3A00&to=2020-08-14+09%3A56%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2522
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A57%3A00&to=2020-08-14+09%3A57%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2523
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+09%3A58%3A00&to=2020-08-14+09%3A58%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2517
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A22%3A00&to=2020-08-14+10%3A22%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2518
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A23%3A00&to=2020-08-14+10%3A23%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2510
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A24%3A00&to=2020-08-14+10%3A24%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2513
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A48%3A00&to=2020-08-14+10%3A48%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2518
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A49%3A00&to=2020-08-14+10%3A49%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2523
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+10%3A50%3A00&to=2020-08-14+10%3A50%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2523
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+11%3A14%3A00&to=2020-08-14+11%3A14%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2526
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+11%3A15%3A00&to=2020-08-14+11%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2534
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+11%3A16%3A00&to=2020-08-14+11%3A16%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2534
DEBUG:urllib3.connectionpool:Starting

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A24%3A00&to=2020-08-14+13%3A24%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2534
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A25%3A00&to=2020-08-14+13%3A25%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2536
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A26%3A00&to=2020-08-14+13%3A26%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2531
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A50%3A00&to=2020-08-14+13%3A50%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2559
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A51%3A00&to=2020-08-14+13%3A51%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2552
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+13%3A52%3A00&to=2020-08-14+13%3A52%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2557
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A16%3A00&to=2020-08-14+14%3A16%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2605
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A17%3A00&to=2020-08-14+14%3A17%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2599
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A18%3A00&to=2020-08-14+14%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2607
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A42%3A00&to=2020-08-14+14%3A42%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2617
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A43%3A00&to=2020-08-14+14%3A43%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2616
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+14%3A44%3A00&to=2020-08-14+14%3A44%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2610
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+15%3A08%3A00&to=2020-08-14+15%3A08%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2629
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+15%3A09%3A00&to=2020-08-14+15%3A09%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2627
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-13+15%3A10%3A00&to=2020-08-14+15%3A10%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2628
DEBUG:urllib3.connectionpool:Starting

In [125]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN1.xlsx')

#### 1 minute interval

In [126]:
market_open = datetime(2020, 8, 14, 9, 30)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:29:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A30%3A00&to=2020-08-14+09%3A30%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6899
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A31%3A00&to=2020-08-14+09%3A31%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6895
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A32%3A00&to=2020-08-14+09%3A32%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6887
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A56%3A00&to=2020-08-14+09%3A56%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6926
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A57%3A00&to=2020-08-14+09%3A57%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6927
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+09%3A58%3A00&to=2020-08-14+09%3A58%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6928
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A22%3A00&to=2020-08-14+10%3A22%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6932
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A23%3A00&to=2020-08-14+10%3A23%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6938
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A24%3A00&to=2020-08-14+10%3A24%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6925
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A48%3A00&to=2020-08-14+10%3A48%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6970
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A49%3A00&to=2020-08-14+10%3A49%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6977
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+10%3A50%3A00&to=2020-08-14+10%3A50%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6965
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+11%3A14%3A00&to=2020-08-14+11%3A14%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6989
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+11%3A15%3A00&to=2020-08-14+11%3A15%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6990
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+11%3A16%3A00&to=2020-08-14+11%3A16%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6994
DEBUG:urllib3.connectionpool:Starting new H

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A24%3A00&to=2020-08-14+13%3A24%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 6987
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A25%3A00&to=2020-08-14+13%3A25%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7000
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A26%3A00&to=2020-08-14+13%3A26%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7002
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A50%3A00&to=2020-08-14+13%3A50%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7057
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A51%3A00&to=2020-08-14+13%3A51%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7057
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+13%3A52%3A00&to=2020-08-14+13%3A52%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7054
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A16%3A00&to=2020-08-14+14%3A16%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7146
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A17%3A00&to=2020-08-14+14%3A17%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7151
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A18%3A00&to=2020-08-14+14%3A18%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7152
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A42%3A00&to=2020-08-14+14%3A42%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7189
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A43%3A00&to=2020-08-14+14%3A43%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7195
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+14%3A44%3A00&to=2020-08-14+14%3A44%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7208
DEBUG:urllib3.connectionpool:Starting new H

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+15%3A08%3A00&to=2020-08-14+15%3A08%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7201
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+15%3A09%3A00&to=2020-08-14+15%3A09%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7201
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/minute?instrument_token=11348226&from=2020-08-13+15%3A10%3A00&to=2020-08-14+15%3A10%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 7189
DEBUG:urllib3.connectionpool:Starting new H

In [127]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN2.xlsx')

#### 5 minute interval Friday

In [128]:
market_open = datetime(2020, 8, 14, 9, 30)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='5minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:29:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A30%3A00&to=2020-08-14+09%3A30%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1550
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A31%3A00&to=2020-08-14+09%3A31%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1550
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A32%3A00&to=2020-08-14+09%3A32%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1551
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A56%3A00&to=2020-08-14+09%3A56%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1548
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A57%3A00&to=2020-08-14+09%3A57%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1546
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+09%3A58%3A00&to=2020-08-14+09%3A58%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1548
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A22%3A00&to=2020-08-14+10%3A22%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1547
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A23%3A00&to=2020-08-14+10%3A23%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1550
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A24%3A00&to=2020-08-14+10%3A24%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1553
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A48%3A00&to=2020-08-14+10%3A48%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1562
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A49%3A00&to=2020-08-14+10%3A49%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1563
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+10%3A50%3A00&to=2020-08-14+10%3A50%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1560
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+11%3A14%3A00&to=2020-08-14+11%3A14%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1553
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+11%3A15%3A00&to=2020-08-14+11%3A15%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1568
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+11%3A16%3A00&to=2020-08-14+11%3A16%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1568
DEBUG:urllib3.connectionpool:Starting

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A24%3A00&to=2020-08-14+13%3A24%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1580
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A25%3A00&to=2020-08-14+13%3A25%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1596
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A26%3A00&to=2020-08-14+13%3A26%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1591
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A50%3A00&to=2020-08-14+13%3A50%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1600
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A51%3A00&to=2020-08-14+13%3A51%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1607
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+13%3A52%3A00&to=2020-08-14+13%3A52%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1600
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A16%3A00&to=2020-08-14+14%3A16%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1610
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A17%3A00&to=2020-08-14+14%3A17%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1618
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A18%3A00&to=2020-08-14+14%3A18%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1617
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A42%3A00&to=2020-08-14+14%3A42%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1625
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A43%3A00&to=2020-08-14+14%3A43%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1622
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+14%3A44%3A00&to=2020-08-14+14%3A44%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1622
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+15%3A08%3A00&to=2020-08-14+15%3A08%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1636
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+15%3A09%3A00&to=2020-08-14+15%3A09%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1638
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-13+15%3A10%3A00&to=2020-08-14+15%3A10%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1637
DEBUG:urllib3.connectionpool:Starting

In [129]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN3.xlsx')

#### 5 minute interval Wednesday

In [130]:
market_open = datetime(2020, 8, 12, 9, 40)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='5minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:15:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+09%3A40%3A00&to=2020-08-12+09%3A40%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1578
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+09%3A41%3A00&to=2020-08-12+09%3A41%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1581
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+09%3A42%3A00&to=2020-08-12+09%3A42%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1576
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A06%3A00&to=2020-08-12+10%3A06%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1577
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A07%3A00&to=2020-08-12+10%3A07%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1583
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A08%3A00&to=2020-08-12+10%3A08%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1587
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A32%3A00&to=2020-08-12+10%3A32%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1580
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A33%3A00&to=2020-08-12+10%3A33%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1582
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A34%3A00&to=2020-08-12+10%3A34%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1579
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A58%3A00&to=2020-08-12+10%3A58%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1570
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+10%3A59%3A00&to=2020-08-12+10%3A59%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1573
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+11%3A00%3A00&to=2020-08-12+11%3A00%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1574
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+11%3A24%3A00&to=2020-08-12+11%3A24%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1572
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+11%3A25%3A00&to=2020-08-12+11%3A25%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1571
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+11%3A26%3A00&to=2020-08-12+11%3A26%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1575
DEBUG:urllib3.connectionpool:Starting

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A08%3A00&to=2020-08-12+13%3A08%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1585
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A09%3A00&to=2020-08-12+13%3A09%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1585
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A10%3A00&to=2020-08-12+13%3A10%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1590
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A34%3A00&to=2020-08-12+13%3A34%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1600
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A35%3A00&to=2020-08-12+13%3A35%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1603
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+13%3A36%3A00&to=2020-08-12+13%3A36%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1619
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A00%3A00&to=2020-08-12+14%3A00%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1604
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A01%3A00&to=2020-08-12+14%3A01%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1602
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A02%3A00&to=2020-08-12+14%3A02%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1604
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A26%3A00&to=2020-08-12+14%3A26%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1610
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A27%3A00&to=2020-08-12+14%3A27%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1609
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A28%3A00&to=2020-08-12+14%3A28%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1604
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A52%3A00&to=2020-08-12+14%3A52%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1617
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A53%3A00&to=2020-08-12+14%3A53%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1620
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/5minute?instrument_token=11348226&from=2020-08-11+14%3A54%3A00&to=2020-08-12+14%3A54%3A00&interval=5minute&continuous=0&oi=0 HTTP/1.1" 200 1617
DEBUG:urllib3.connectionpool:Starting

In [131]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN4.xlsx')

#### 10 minute interval Wednesday

In [132]:
market_open = datetime(2020, 8, 12, 9, 40)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='10minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:15:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+09%3A40%3A00&to=2020-08-12+09%3A40%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+09%3A41%3A00&to=2020-08-12+09%3A41%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+09%3A42%3A00&to=2020-08-12+09%3A42%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A06%3A00&to=2020-08-12+10%3A06%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A07%3A00&to=2020-08-12+10%3A07%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A08%3A00&to=2020-08-12+10%3A08%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A32%3A00&to=2020-08-12+10%3A32%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A33%3A00&to=2020-08-12+10%3A33%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A34%3A00&to=2020-08-12+10%3A34%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A58%3A00&to=2020-08-12+10%3A58%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+10%3A59%3A00&to=2020-08-12+10%3A59%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+11%3A00%3A00&to=2020-08-12+11%3A00%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+11%3A24%3A00&to=2020-08-12+11%3A24%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+11%3A25%3A00&to=2020-08-12+11%3A25%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+11%3A26%3A00&to=2020-08-12+11%3A26%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A08%3A00&to=2020-08-12+13%3A08%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A09%3A00&to=2020-08-12+13%3A09%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A10%3A00&to=2020-08-12+13%3A10%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A34%3A00&to=2020-08-12+13%3A34%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A35%3A00&to=2020-08-12+13%3A35%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+13%3A36%3A00&to=2020-08-12+13%3A36%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A00%3A00&to=2020-08-12+14%3A00%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A01%3A00&to=2020-08-12+14%3A01%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A02%3A00&to=2020-08-12+14%3A02%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A26%3A00&to=2020-08-12+14%3A26%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A27%3A00&to=2020-08-12+14%3A27%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A28%3A00&to=2020-08-12+14%3A28%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A52%3A00&to=2020-08-12+14%3A52%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A53%3A00&to=2020-08-12+14%3A53%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-11+14%3A54%3A00&to=2020-08-12+14%3A54%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

In [133]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN5.xlsx')

#### 10 minute interval Friday

In [134]:
market_open = datetime(2020, 8, 14, 9, 40)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='10minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:29:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+09%3A40%3A00&to=2020-08-14+09%3A40%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+09%3A41%3A00&to=2020-08-14+09%3A41%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+09%3A42%3A00&to=2020-08-14+09%3A42%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A06%3A00&to=2020-08-14+10%3A06%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A07%3A00&to=2020-08-14+10%3A07%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A08%3A00&to=2020-08-14+10%3A08%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A32%3A00&to=2020-08-14+10%3A32%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A33%3A00&to=2020-08-14+10%3A33%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A34%3A00&to=2020-08-14+10%3A34%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A58%3A00&to=2020-08-14+10%3A58%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+10%3A59%3A00&to=2020-08-14+10%3A59%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+11%3A00%3A00&to=2020-08-14+11%3A00%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+11%3A24%3A00&to=2020-08-14+11%3A24%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+11%3A25%3A00&to=2020-08-14+11%3A25%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+11%3A26%3A00&to=2020-08-14+11%3A26%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+13%3A34%3A00&to=2020-08-14+13%3A34%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+13%3A35%3A00&to=2020-08-14+13%3A35%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+13%3A36%3A00&to=2020-08-14+13%3A36%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A00%3A00&to=2020-08-14+14%3A00%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A01%3A00&to=2020-08-14+14%3A01%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A02%3A00&to=2020-08-14+14%3A02%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A26%3A00&to=2020-08-14+14%3A26%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A27%3A00&to=2020-08-14+14%3A27%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A28%3A00&to=2020-08-14+14%3A28%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A52%3A00&to=2020-08-14+14%3A52%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A53%3A00&to=2020-08-14+14%3A53%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+14%3A54%3A00&to=2020-08-14+14%3A54%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+15%3A18%3A00&to=2020-08-14+15%3A18%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+15%3A19%3A00&to=2020-08-14+15%3A19%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-13+15%3A20%3A00&to=2020-08-14+15%3A20%3A00&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:St

In [135]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN6.xlsx')

#### 1 minute

In [29]:
market_open = datetime(2020, 9, 4, 9, 18)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
for delta in range(360):
    sleep(0.5)
    test_end = market_open + timedelta(minutes=delta)
    print(test_end)
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval='3minute')
    smi_df = get_smi(hist_data, 10, 3, 3, 10)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")=='15:27:00':
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+09%3A18%3A00&to=2020-09-04+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2556
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 09:19:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+09%3A19%3A00&to=2020-09-04+09%3A19%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2552
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+15%3A15%3A00&to=2020-09-04+15%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2630
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 15:16:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+15%3A16%3A00&to=2020-09-04+15%3A16%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2630
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 15:17:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+15%3A17%3A00&to=2020-09-04+15%3A17%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2634


In [30]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log_20200904_3min.xlsx')

# Zerodha LIVE Test Deployment

#### 10 minute interval LIVE

In [ ]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20AUGFUT'
instrument_type = 'FUT'
exchange = 'NFO'


# CHART SPECIFICATIONS
candle_size = '10minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=17, hour=15 , minute=30 , second=0)

In [27]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
while datetime.strftime(datetime.now(), "%H:%M:%S")!=datetime.strftime(deploy_till, "%H:%M:%S"):
    sleep(0.5)
    test_end = datetime.now()
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
    smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")==datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M:%S"):
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A13%3A44&to=2020-08-17+11%3A13%3A44&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
INFO:numexpr.utils:NumExpr defaulting to 4 threads.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A13%3A58&to=2020-08-17+11%3A13%3A58&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A14%3A00&to=2020-08-17+11%3A14%3A00&interval=10minute&continuous=0&oi=

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A14%3A53&to=2020-08-17+11%3A14%3A53&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A14%3A54&to=2020-08-17+11%3A14%3A54&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A14%3A55&to=2020-08-17+11%3A14%3A55&interval=10minute&continuous=0&oi=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


ConnectTimeout: HTTPSConnectionPool(host='api.kite.trade', port=443): Max retries exceeded with url: /instruments/historical/11348226/10minute?instrument_token=11348226&from=2020-08-17+09%3A15%3A08&to=2020-08-17+11%3A15%3A08&interval=10minute&continuous=0&oi=0 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x00000177E27A52C8>, 'Connection to api.kite.trade timed out. (connect timeout=7)'))

In [28]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\Trade_Log-RUN6.xlsx')

# Zerodha LIVE Deployment TradeLog

In [21]:
def place_trade_and_manage_log(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type, complete_list, reject_list):
    current_order_id = kite.place_order(tradingsymbol=tradingsymbol, exchange=exchange, quantity=qty, product=product, order_type=order_type, transaction_type=current_transaction_type, variety=variety)
    while True:
        sleep(5)
        current_order_details = kite.order_history(current_order_id)[-1]
        if current_order_details['status']=='COMPLETE':
            if len(trade_log)==0 or len(trade_log)%2==0:
                current_status = 'open'
            else:
                current_status = 'closed'
            trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
            complete_list.append(current_order_details)
            break
        elif current_order_details['status']=='CANCELLED' or current_order_details['status']=='REJECTED':
            reject_list.append(current_order_details)
            break
        else:
            continue
    return trade_log, current_transaction_type, current_status, complete_list, reject_list

# Code for Zerodha Deployment

In [95]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20AUGFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '10minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=17, hour=15 , minute=30 , second=0)

In [ ]:
#instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
complete_list = []
reject_list = []
while datetime.strftime(datetime.now(), "%H:%M:%S")!=datetime.strftime(deploy_till, "%H:%M:%S"):
    sleep(0.5)
    test_end = datetime.now()
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
    smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H:%M:%S")==datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M:%S"):
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
                break
        else:
            break
    elif current_smi>current_signal:
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
    elif current_smi<current_signal:
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
            trade_log, current_transaction_type, current_status, complete_list, reject_list = place_trade_and_manage_log(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type, complete_list=complete_list, reject_list=reject_list)
        elif current_status=='open' and current_transaction_type=='sell':
            continue

# Signal Follower with attempt at Profit Maximization

In [21]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

In [22]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [23]:
instoken = get_instrument_token(tradingsymbol,instrument_type,exchange)

NameError: name 'tradingsymbol' is not defined

In [ ]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
signal_buffer = 4
trend_change_buffer = 1

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)
candle = 3

In [25]:
market_open_list = [datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 25, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15), datetime(2020, 9, 7, 9, 15), datetime(2020, 9, 8, 9, 15), datetime(2020, 9, 9, 9, 15)]
previous_close_list = [22310, 22800, 23075, 23435, 23613, 24622, 23660, 23945, 23920, 23579, 23084, 23020, 22657]
#market_open_list = []
#previous_close_list = []

In [105]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    previous_close = previous_close_list[i]
    trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
    smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
    current_status='closed'
    benchmark_gap = 0
    first_trade = True
    gap_up = False
    gap_down = False
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=3)
        for _ in range(5):
                try:
                    hist_data = get_historical_data(instoken, test_start, test_end, interval)
                    break
                except requests.exceptions.ReadTimeout as e:
                    logging.exception("Timeout Error - while getting ltp: \n %s", e)
                    logging.info("Retrying")
                    continue
        
        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-2]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-3], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]
        
        if current_time==(datetime.strftime(market_open, "%Y-%m-%d")+" 09:15:00"):
            if previous_close+100<list(hist_data['close'])[-2]:
                gap_up = True
                gup_points = list(hist_data['close'])[-2]-previous_close
            if previous_close-100>list(hist_data['close'])[-2]:
                gap_down = True
                gdown_points = previous_close-list(hist_data['close'])[-2]
        else:
            if gap_up:
                print("GAP UP!")
                if list(hist_data['low'])[-2]<(previous_close+(0.25*gup_points)):
                    gap_up = False
            elif gap_down:
                print("GAP DOWN!")
                if list(hist_data['high'])[-2]>(previous_close-(0.25*gdown_points)):
                    gap_down = False
            else:
                print("NO LONGER GAP UP or DOWN")
        
        # Exit at end of day
        if datetime.strftime(test_end, "%H:%M:%S")=='15:18:00':
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        if first_trade:
            if current_status=="closed":
                if current_gap>0 and not gap_down:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                elif current_gap<0 and not gap_up:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
            elif current_status=="open":
                if current_transaction_type=="buy":
                    if current_gap<0 and abs(last_signal-current_signal)>trend_change_buffer:
                        transaction_type='sell'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        if not gap_up:
                            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                            trend_gap = current_gap
                            benchmark_gap = current_gap
                        last_signal = current_signal
                        first_trade = False
                    elif current_gap>0:
                        last_signal = current_signal
                elif current_transaction_type=="sell":
                    if current_gap>0 and abs(last_signal-current_signal)>trend_change_buffer:
                        transaction_type='buy'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        if not gap_down:
                            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                            trend_gap = current_gap
                            benchmark_gap = current_gap
                        last_signal = current_signal
                        first_trade = False
                    elif current_gap<0:
                        last_signal = current_signal
        
        if not first_trade:
            if current_status=="open":
                if current_transaction_type=="buy":
                    if (current_gap<0 and abs(last_signal-current_signal)>trend_change_buffer):
                        transaction_type='sell'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        if not gap_up:
                            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                            trend_gap = current_gap
                            benchmark_gap = current_gap
                        last_signal = current_signal
                    elif (current_gap>0 and abs(benchmark_gap-current_gap)>signal_buffer and current_gap<benchmark_gap):
                        transaction_type='sell'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        benchmark_gap = 0
                        last_signal = current_signal
                    elif current_gap>benchmark_gap:
                        benchmark_gap = current_gap
                        last_signal = current_signal
                    elif current_gap>0:
                        last_signal = current_signal
                elif current_transaction_type=="sell":
                    if (current_gap>0 and abs(last_signal-current_signal)>trend_change_buffer):
                        transaction_type='buy'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        if not gap_down:
                            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                            trend_gap = current_gap
                            benchmark_gap = current_gap
                        last_signal = current_signal
                    elif (current_gap<0 and abs(benchmark_gap-current_gap)>signal_buffer and current_gap>benchmark_gap):
                        transaction_type='buy'
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                        last_signal = current_signal
                        benchmark_gap = 0
                    elif current_gap<benchmark_gap:
                        benchmark_gap = current_gap
                        last_signal = current_signal
                    elif current_gap<0:
                        last_signal = current_signal
            if current_status=="closed":
                if current_gap>0 and trend_gap<0 and abs(last_signal-current_signal)>trend_change_buffer and not gap_down: 
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    benchmark_gap = current_gap
                    trend_gap = current_gap
                    last_signal = current_signal
                elif current_gap<0 and trend_gap>0 and abs(last_signal-current_signal)>trend_change_buffer and not gap_up:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    benchmark_gap = current_gap
                    trend_gap = current_gap
                    last_signal = current_signal
                elif current_gap*trend_gap>0:
                    last_signal = current_signal
    
    trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'_GUDA.xlsx')
    smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'_GUDA.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A18%3A00&to=2020-08-24+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2260
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:21:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A21%3A00&to=2020-08-24+09%3A21%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2274


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-22+11%3A00%3A00&to=2020-08-25+11%3A00%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 3144


NO LONGER GAP UP or DOWN


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-25 11:03:00


KeyboardInterrupt: 

# Layman Signal Following

In [25]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [26]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
signal_buffer = 4
trend_change_buffer = 1

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)
candle = 3

In [29]:
#market_open_list = [datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15), datetime(2020, 9, 7, 9, 15), datetime(2020, 9, 8, 9, 15), datetime(2020, 9, 9, 9, 15)]
#previous_close_list = [22310, 23075, 23435, 23613, 24622, 23660, 23945, 23920, 23579, 23084, 23020, 22657]
market_open_list = [datetime(2020, 9, 24, 9, 15)]
previous_close_list = [21301]

In [ ]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    previous_close = previous_close_list[i]
    trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
    smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
    current_status='closed'
    last_signal = 0
    gap_up = False
    gap_down = False
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=3)
        for _ in range(5):
                try:
                    hist_data = get_historical_data(instoken, test_start, test_end, interval)
                    break
                except requests.exceptions.ReadTimeout as e:
                    logging.exception("Timeout Error - while getting ltp: \n %s", e)
                    logging.info("Retrying")
                    continue
        
        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-2]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-3], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]
        
        if current_time==(datetime.strftime(market_open, "%Y-%m-%d")+" 09:15:00"):
            if previous_close+100<list(hist_data['open'])[-2]:
                gap_up = True
                gup_points = list(hist_data['open'])[-2]-previous_close
            if previous_close-100>list(hist_data['open'])[-2]:
                gap_down = True
                gdown_points = previous_close-list(hist_data['open'])[-2]
        
        # Exit at end of day
        if datetime.strftime(test_end, "%H:%M:%S")=='15:18:00':
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        if gap_up:
            if list(hist_data['low'])[-2]<(previous_close+(0.25*gup_points)):
                gap_up = False
        if gap_down:
            if list(hist_data['high'])[-2]>(previous_close-(0.25*gdown_points)):
                gap_down = False
        
        if current_status=="closed":
            if current_gap>0 and not gap_down and abs(last_signal-current_signal)>trend_change_buffer:
                transaction_type='buy'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                trend_gap = current_gap
                last_signal = current_signal
            elif current_gap<0 and not gap_up and abs(last_signal-current_signal)>trend_change_buffer:
                transaction_type='sell'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                trend_gap = current_gap
                last_signal = current_signal
        elif current_status=="open":
            if current_transaction_type=="buy":
                if current_gap<0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                elif current_gap>0:
                    last_signal = current_signal
            if current_transaction_type=="sell":
                if current_gap>0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                elif current_gap<0:
                    last_signal = current_signal
    
    trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'.xlsx')
    smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A18%3A00&to=2020-08-24+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2260
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:21:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A21%3A00&to=2020-08-24+09%3A21%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2274
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

2020-08-24 13:09:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+13%3A09%3A00&to=2020-08-24+13%3A09%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2434
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 13:12:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+13%3A12%3A00&to=2020-08-24+13%3A12%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2438
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 13:15:00


In [99]:
trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'.xlsx')
smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\With_Profit_Taking_and_GapUpDown_Adjustment\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(signal_buffer)+'.xlsx')

# Layman Signal Following with Stoploss and 30 point profit booking

In [22]:
def calculate_pnl(current_transaction_type, transaction_price, price):
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    return pnl

In [23]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [24]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20OCTFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
trend_change_buffer = 1
stop_loss = -80
profit = 30

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)
candle = 3

In [28]:
market_open_list = [datetime(2020, 10, 5, 9, 15), datetime(2020, 10, 6, 9, 15), datetime(2020, 10, 7, 9, 15), datetime(2020, 10, 8, 9, 15), datetime(2020, 10, 9, 9, 15), datetime(2020, 10, 12, 9, 15), datetime(2020, 10, 13, 9, 15), datetime(2020, 10, 14, 9, 15), datetime(2020, 10, 15, 9, 15), datetime(2020, 10, 16, 9, 15), datetime(2020, 10, 19, 9, 15), datetime(2020, 10, 20, 9, 15)]
previous_close_list = [22292, 22443, 22909, 22999, 23163, 23951, 23769, 23500, 23940, 23036, 23579, 24325]

#market_open_list = [datetime(2020, 8, 18, 9, 15), datetime(2020, 8, 19, 9, 15), datetime(2020, 8, 20, 9, 15), datetime(2020, 8, 21, 9, 15), datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 25, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15), datetime(2020, 9, 7, 9, 15), datetime(2020, 9, 8, 9, 15), datetime(2020, 9, 9, 9, 15), datetime(2020, 9, 10, 9, 15), datetime(2020, 9, 11, 9, 15), datetime(2020, 9, 14, 9, 15), datetime(2020, 9, 15, 9, 15), datetime(2020, 9, 16, 9, 15), datetime(2020, 9, 17, 9, 15), datetime(2020, 9, 18, 9, 15), datetime(2020, 9, 21, 9, 15), datetime(2020, 9, 22, 9, 15), datetime(2020, 9, 23, 9, 15), datetime(2020, 9, 25, 9, 15), datetime(2020, 9, 28, 9, 15), datetime(2020, 9, 29, 9, 15), datetime(2020, 9, 30, 9, 15), datetime(2020, 10, 1, 9, 15), datetime(2020, 10, 5, 9, 15), datetime(2020, 10, 6, 9, 15), datetime(2020, 10, 7, 9, 15), datetime(2020, 10, 8, 9, 15), datetime(2020, 10, 9, 9, 15), datetime(2020, 10, 12, 9, 15), datetime(2020, 10, 13, 9, 15), datetime(2020, 10, 14, 9, 15), datetime(2020, 10, 15, 9, 15), datetime(2020, 10, 16, 9, 15), datetime(2020, 10, 19, 9, 15), datetime(2020, 10, 20, 9, 15)]
#previous_close_list = [21801, 22206, 22325, 21939, 22310, 22800, 23075, 23435, 23613, 24622, 23660, 23945, 23920, 23579, 23084, 23020, 22657, 22375, 22533, 22575, 22154, 22597, 22646, 22391, 22121, 21292, 21222, 20534, 20935, 21753, 21487, 21435, 22292, 22443, 22909, 22999, 23163, 23951, 23769, 23500, 23940, 23036, 23579, 24325]

In [29]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    previous_close = previous_close_list[i]
    trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
    smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
    current_status='closed'
    last_signal = 0
    trend_gap = 0
    gap_up = False
    gap_down = False
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=4)
        for _ in range(5):
                try:
                    hist_data = get_historical_data(instoken, test_start, test_end, interval)
                    break
                except requests.exceptions.ReadTimeout as e:
                    logging.exception("Timeout Error - while getting ltp: \n %s", e)
                    logging.info("Retrying")
                    continue
        
        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-2]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-3], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]
        
        # Exit at end of day
        if datetime.strftime(test_end, "%H:%M:%S")=='15:18:00':
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        if current_gap>0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        elif current_gap<0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif current_gap*trend_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        
        if current_status=="open":
            if current_transaction_type=="buy":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['high'])[-2])
            elif current_transaction_type=="sell":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['low'])[-2])
            if pnl<stop_loss:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['high'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['low'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
            elif pnl>profit:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['high'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['low'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
    
    trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\With_Profit_Booking_and_stoploss\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(profit)+'_Stoploss.xlsx')
    smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\With_Profit_Booking_and_stoploss\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_'+str(profit)+'_Stoploss.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-05 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-01+09%3A18%3A00&to=2020-10-05+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2616
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-05 09:21:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-01+09%3A21%3A00&to=2020-10-05+09%3A21%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2616
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-16+15%3A12%3A00&to=2020-10-20+15%3A12%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 5089
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-20 15:15:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-16+15%3A15%3A00&to=2020-10-20+15%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 5078
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-20 15:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-16+15%3A18%3A00&to=2020-10-20+15%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 5082


# Layman Signal Following with Stoploss, GUDA and Overbought/Oversold entry

In [26]:
def calculate_pnl(current_transaction_type, transaction_price, price):
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    return pnl

In [27]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [28]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
trend_change_buffer = 1
stop_loss = -100
profit = 30
overbought = 35
oversold = -35

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)
candle = 3

In [35]:
market_open_list = [datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 25, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15), datetime(2020, 9, 7, 9, 15), datetime(2020, 9, 8, 9, 15), datetime(2020, 9, 9, 9, 15)]
previous_close_list = [22310, 22800, 23075, 23435, 23613, 24622, 23660, 23945, 23920, 23579, 23084, 23020, 22657]
#market_open_list = []
#previous_close_list = []

In [36]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    previous_close = previous_close_list[i]
    trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
    smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
    current_status='closed'
    last_signal = 0
    trend_gap = 0
    gap_up = False
    gap_down = False
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=3)
        for _ in range(5):
                try:
                    hist_data = get_historical_data(instoken, test_start, test_end, interval)
                    break
                except requests.exceptions.ReadTimeout as e:
                    logging.exception("Timeout Error - while getting ltp: \n %s", e)
                    logging.info("Retrying")
                    continue
        
        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-2]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-3], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]
        
        if current_time==(datetime.strftime(market_open, "%Y-%m-%d")+" 09:15:00"):
            if previous_close+100<list(hist_data['close'])[-2]:
                gap_up = True
                gup_points = list(hist_data['close'])[-2]-previous_close
            if previous_close-100>list(hist_data['close'])[-2]:
                gap_down = True
                gdown_points = previous_close-list(hist_data['close'])[-2]
        else:
            if gap_up:
                print("GAP UP!")
                if list(hist_data['low'])[-2]<(previous_close+(0.25*gup_points)):
                    gap_up = False
            elif gap_down:
                print("GAP DOWN!")
                if list(hist_data['high'])[-2]>(previous_close-(0.25*gdown_points)):
                    gap_down = False
            else:
                print("NO LONGER GAP UP or DOWN")
        
        # Exit at end of day
        if datetime.strftime(test_end, "%H:%M:%S")=='15:18:00':
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        if current_gap>0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer and (current_signal>35 or current_signal<-35):
                    transaction_type='buy'
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                    if not gap_down and (current_signal>35 or current_signal<-35):
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        elif current_gap<0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer and (current_signal>35 or current_signal<-35):
                    transaction_type='sell'
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif current_gap*trend_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                    if not gap_up and (current_signal>35 or current_signal<-35):
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        
        if current_status=="open":
            if current_transaction_type=="buy":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['high'])[-2])
            elif current_transaction_type=="sell":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['low'])[-2])
            if pnl<stop_loss:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['high'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['low'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
    
    trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\Without_Profit_Booking_with_GUDA_stoploss_OBOSEntry\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_Stoploss.xlsx')
    smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\Without_Profit_Booking_with_GUDA_stoploss_OBOSEntry\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_Stoploss.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A18%3A00&to=2020-08-24+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2260
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-24 09:21:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-21+09%3A21%3A00&to=2020-08-24+09%3A21%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2274


...

GAP DOWN!


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-09 15:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-06+15%3A18%3A00&to=2020-09-09+15%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 7562


GAP DOWN!


# Eureka ?

In [21]:
def update_trade_log(trade_log, profit, stop_loss):
    pnl, cap, hour, pol, max_pnl = ([], [], [], [], [])
    total_pnl = 0
    for i in list(trade_log.index):
        if trade_log.loc[i, 'status']=='open':
            if trade_log.loc[i, 'transaction_type']=='buy':
                pnl_val = trade_log.loc[i+1, 'price']-trade_log.loc[i, 'price']
            elif trade_log.loc[i, 'transaction_type']=='sell':
                pnl_val = trade_log.loc[i, 'price']-trade_log.loc[i+1, 'price']
        else:
            pnl_val = 0
        pnl.append(pnl_val)
        
        if pnl_val==0:
            cap.append(0)
            pol.append("")
            max_pnl.append(total_pnl)
        else:
            if pnl_val>0:
                if pnl_val>profit:
                    cap_val = profit
                else:
                    cap_val = pnl_val
                pol.append('P')
            elif pnl_val<0:
                if pnl_val<stop_loss:
                    cap_val = stop_loss
                else:
                    cap_val = pnl_val
                pol.append('L')
            total_pnl+=cap_val
            max_pnl.append(total_pnl)
            cap.append(cap_val)
        
        hour.append(trade_log.loc[i, 'time'][11:13])
        
    trade_log['PnL'] = pnl
    trade_log['CAP'] = cap
    trade_log['HOUR'] = hour
    trade_log['Profit or Loss'] = pol
    trade_log['Max PnL'] = max_pnl
    return trade_log

In [22]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_price, current_transaction_type, current_status

In [23]:
def calculate_pnl(current_transaction_type, transaction_price, price):
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    return pnl

In [26]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20OCTFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
interval = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10
trend_change_buffer = 1
stop_loss = -80
profit = 30

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)
candle = 3

In [25]:
def update_summary(summary, trade_log, interval, market_open, profit, stop_loss):
    pnl = trade_log.loc[list(trade_log.index)[-1], 'Max PnL']
    trade_count = len(list(trade_log['Max PnL']))/2
    net_pnl = pnl - (trade_count*5.5)
    cash = net_pnl*25
    summary.loc[len(summary)] = [interval, datetime.strftime(market_open, "%d-%m-%Y"), 'Signal Follow - With '+str(profit)+' profit taking, '+str(stop_loss)+' stoploss and GUDA', 'n/a', pnl, trade_count, net_pnl, cash]
    return summary

In [27]:
market_open_list = [datetime(2020, 10, 1, 9, 15), datetime(2020, 10, 5, 9, 15), datetime(2020, 10, 6, 9, 15), datetime(2020, 10, 7, 9, 15), datetime(2020, 10, 8, 9, 15), datetime(2020, 10, 9, 9, 15), datetime(2020, 10, 12, 9, 15), datetime(2020, 10, 13, 9, 15), datetime(2020, 10, 14, 9, 15), datetime(2020, 10, 15, 9, 15), datetime(2020, 10, 16, 9, 15), datetime(2020, 10, 19, 9, 15), datetime(2020, 10, 20, 9, 15), datetime(2020, 10, 21, 9, 15), datetime(2020, 10, 22, 9, 15), datetime(2020, 10, 23, 9, 15)]
previous_close_list = [21435, 22292, 22443, 22909, 22999, 23163, 23951, 23769, 23500, 23940, 23036, 23579, 24325, 24347, 24699, 24563]

#market_open_list = [datetime(2020, 10, 20, 9, 15)]
#previous_close_list = [24325]

In [28]:
summary = pd.DataFrame(columns=['CANDLE', 'DATE','THRESHOLD','Signal_Buffer','PNL','TRADE COUNT','NET PNL','CASH PnL'])
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    previous_close = previous_close_list[i]
    trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
    smi_log = pd.DataFrame(columns=['time','price','smi','signal','signal_gap'])
    current_status='closed'
    last_signal = 0
    trend_gap = 0
    gap_up = False
    gap_down = False
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=4)
        for _ in range(5):
                try:
                    hist_data = get_historical_data(instoken, test_start, test_end, interval)
                    break
                except requests.exceptions.ReadTimeout as e:
                    logging.exception("Timeout Error - while getting ltp: \n %s", e)
                    logging.info("Retrying")
                    continue
                except:
                    continue

        smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
        current_time = list(smi_df.index)[-2]
        current_price = smi_df.loc[current_time, 'close']
        current_smi = smi_df.loc[current_time, 'smi']
        current_signal = smi_df.loc[current_time, 'signal']
        current_gap = current_signal- smi_df.loc[list(smi_df.index)[-3], 'signal']
        smi_log.loc[len(smi_log)] = [current_time, current_price, current_smi, current_signal, current_gap]

        if current_time==(datetime.strftime(market_open, "%Y-%m-%d")+" 09:15:00"):
            if previous_close+100<list(hist_data['close'])[-2]:
                gap_up = True
                gup_points = list(hist_data['close'])[-2]-previous_close
            if previous_close-100>list(hist_data['close'])[-2]:
                gap_down = True
                gdown_points = previous_close-list(hist_data['close'])[-2]
        else:
            if gap_up:
                print("GAP UP!")
                if list(hist_data['low'])[-2]<(previous_close+(0.25*gup_points)):
                    gap_up = False
            elif gap_down:
                print("GAP DOWN!")
                if list(hist_data['high'])[-2]>(previous_close-(0.25*gdown_points)):
                    gap_down = False
            else:
                print("NO LONGER GAP UP or DOWN")

        # Exit at end of day
        if datetime.strftime(test_end, "%H:%M:%S")=='15:12:00':
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
                elif current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    break
            else:
                break

        if current_status=="open":
            if current_transaction_type=="buy":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['high'])[-2])
            elif current_transaction_type=="sell":
                pnl = calculate_pnl(current_transaction_type, transaction_price, list(hist_data['low'])[-2])
            if pnl<stop_loss:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['high'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['low'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
            elif pnl>profit:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['high'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=list(hist_data['low'])[-2], current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
            else:
                if current_transaction_type=="buy":
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                elif current_transaction_type=="sell":
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal

        if current_gap>0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='buy'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    trend_gap = current_gap
                    last_signal = current_signal
                    if not gap_down:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal
        elif current_gap<0:
            if current_status=="closed":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                elif current_gap*trend_gap>=0:
                    last_signal = current_signal
            elif current_status=="open":
                if trend_gap*current_gap<=0 and abs(last_signal-current_signal)>trend_change_buffer:
                    transaction_type='sell'
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                    last_signal = current_signal
                    trend_gap = current_gap
                    if not gap_up:
                        trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                elif trend_gap*current_gap>=0:
                    last_signal = current_signal

    update_trade_log(trade_log, profit, stop_loss)
    update_summary(summary, trade_log, interval, market_open, profit, stop_loss)
    trade_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\Others\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_SL_'+str(stop_loss)+'_Profit_'+str(profit)+'.xlsx')
    smi_log.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\SMI_Logs\Others\SMI_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'_SL_'+str(stop_loss)+'_Profit_'+str(profit)+'.xlsx')
summary.to_excel('Stochastic_Momentum_Indicator-Test_Outputs\_'+interval+'\Signal_Follower\Trade_Logs\Others\Summary.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-01 09:18:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-09-27+09%3A18%3A00&to=2020-10-01+09%3A18%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 7707
INFO:numexpr.utils:NumExpr defaulting to 4 threads.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-01 09:21:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-09-27+09%3A21%3A00&to=2020-10-01+09%3A21%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 7731


...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-10-16 11:51:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/13014018/3minute?instrument_token=13014018&from=2020-10-12+11%3A51%3A00&to=2020-10-16+11%3A51%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 10203


GAP UP!


KeyboardInterrupt: 